# Euclidean Heat Kernel - Diffusion & Denoising

In this notebook, we will show how we can
- use the exact score function for the Euclidean heat kernel to reverse the forward diffusion process
- train a neural network to approximate this score function and replicate the same denoising procedure
  
The idea is that we can have a direct comparison between our ML results and the analytical results, since we know the score function for the heat kernel exactly.

*Note: In more practical applications when we start with non-trivial 'training data' (i.e., give the heat equation initial conditions), we do not know the true score function for all time.*

## Setup

In [ ]:
# General imports
import numpy as np
import torch

import tqdm.auto as tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
# Imports from our repo
import sys
sys.path.insert(0, '..')  # repo source code

from src.diffusion import VarianceExpandingDiffusion
from src.heat import eucl_score_hk
from src.utils import grab, wrap
from src.devices import set_device, get_device, summary

In [ ]:
set_device('cuda', 0)
print(summary())

## Background: Euclidean Diffusion

Variance-expanding diffusion is a stochastic process described by $$dx = g(t) dW$$ that corresponds to the heat equation: $$\partial_t u(x, t) = \frac{g(t)^2}{2} \Delta u(x, t),$$ where $g^2$ is a postive, time-dependent scalar quantity we call the *diffusivity*. The differential operator $\Delta$ is the Laplace-Beltrami operator on whatever ambient space the samples $x$ occupy. In Euclidean space, $x \in \mathbb{R}^n$, $\Delta$ is just the familiar Laplacian given by $\Delta := \sum_{i=1}^n \partial_i^2$. In particuar, for a single, real-valued degree of freedom (with constant unit diffusivity), the heat equation is $$\partial_t u = \partial_x^2 u,$$ and the fundamental solution (Green's function / propagator) for this PDE is called the *Heat Kernel* and is given by $$K(x, t) = \frac{1}{\sqrt{2\pi\sigma(t)^2}}e^{-\frac{x^2}{2\sigma(t)^2}},$$ where $\sigma(t)$ is the marginal standard deviation

In [ ]:
def euclidean_heat_kernel(x, t, width=None):
    """Computes the Euclidean heat kernel density K(x, t) for `x` at time `t`."""
    if width is None:
        width = t**0.5  # assume unit constant diffusivity
    normalization = 1 / (2 * np.pi * width**2)**0.5
    weight = torch.exp(-x**2 / (2 * width**2))
    return normalization * weight


def _visualize_heat_kernel():
    xs = torch.linspace(-4, 4, 100)
    times = np.linspace(0.05, 1, 20)
    
    fig, ax = plt.subplots(1, 1)
    cmap = mpl.colormaps.get_cmap('viridis')
    ax.set_xlabel('$x$')
    ax.set_ylabel('Heat Kernel Density')
    for t in times:
        line = ax.plot(grab(xs), grab(euclidean_heat_kernel(xs, t)), color=cmap(t))#, label=f'$t = {t}$')
    time_colors = mpl.cm.ScalarMappable(mpl.colors.Normalize(times[0], times[-1]))
    cbar = fig.colorbar(time_colors, cmap=cmap, ax=ax, label='$t$')
    fig.show()

_visualize_heat_kernel()

One can easily sample from the Euclidean heat kernel at arbitrary time $t$; since $K(x, t)$ is always a normal distribution, you only need to know the marginal standard deviation at $t$.

In [ ]:
def sample_euclid_hk(batch_size, t, width=None):
    """Generates `batch_size` samples from the Euclidean
    Heat kernel at time `t` with width `width`."""
    if width is None:
        width = t**0.5
    x_t = width * torch.randn((batch_size,))
    return x_t

def _test_sample_hk():
    batch_size = 16384
    xs = torch.linspace(-4, 4, 100)
    times = [0.1, 0.5, 1.0]
    
    cmap = plt.get_cmap('viridis')
    fig, ax = plt.subplots(1, 1)
    ax.set_xlabel('$x$')
    ax.set_ylabel('Density')
    for t in times:
        xt = sample_euclid_hk(batch_size, t)
        ax.hist(grab(xt), bins=50, density=True, color=cmap(t), alpha=0.55, label=f'Heat kernel samples at $t = {t}$')
        ax.plot(grab(xs), grab(euclidean_heat_kernel(xs, t)), ls='--', color=cmap(t), label=f'Analytical HK at $t = {t}$')
    fig.legend(loc='right', fontsize='small')
    ax.set_title('Heat Kernel Density')
    fig.show()

_test_sample_hk()

It's also easy to simulate the forward process by directly feeding initial data (zero) into our diffuser object:

In [ ]:
def visualize_euclid_fwd():
    """Simulates forward Euclidean VE diffusion process."""
    batch_size = 4096
    x_0 = torch.zeros((batch_size, 1))
    diffuser = VarianceExpandingDiffusion(sigma=1.1)
    
    xs = torch.linspace(-4, 4, 100)
    times = [0.01, 0.1, 0.5, 1.0]
    cmap = mpl.colormaps.get_cmap('viridis')
    
    fig, axes = plt.subplots(1, len(times), figsize=(4*len(times), 4), sharey=True)
    fig.suptitle('Density During Forward Process')
    axes[0].set_ylabel('Density')
    for t, ax in zip(times, axes):
        # Samples stats should match params of analytical HK
        x_t = diffuser(x_0, torch.tensor(t).repeat(batch_size))
        sigma_t = diffuser.sigma_func(t)
        assert torch.allclose(torch.std(x_t), sigma_t, atol=5e-2), \
            f'StDev of samples {torch.std(x_t).item():.4f} does not match marginal HK StDev {sigma_t:.4f}'
        
        # Visualize forward process over time
        ax.set_title(f'$t = {t}$')
        ax.set_xlabel('$x$')
        ax.hist(grab(x_t), bins=50, density=True, color=cmap(t), alpha=0.55, label=f'Diffused samples at $t = {t}$')
        ax.plot(grab(xs), grab(euclidean_heat_kernel(xs, t, width=sigma_t)), ls='--', color=cmap(t), label=f'Analytical HK at $t = {t}$')
        ax.legend()
    fig.show()

visualize_euclid_fwd()

## Denoising with the Exact Score Function

The analytical score function for the Euclidean heat kernel is simply given by:

$$s(x, t) := \partial_x \log K(x, t) = -\frac{x}{\sigma(t)^2}$$

which can be used to solve either the ODE or SDE in reverse time: $$d\tilde{x} = -g(t)^2 s(x, t)dt + g(t) dW$$ $$d\tilde{x} = -\frac{1}{2}g(t)^2 s(x, t)dt$$

In [ ]:
def denoise_backward(x_1, diffuser, num_steps=200, solver_type='ODE'):
    dt = 1 / num_steps
    t = 1.0
    x_t = x_1.clone()

    trajectories = []
    for step in range(num_steps):
        sigma_t = diffuser.sigma_func(t)
        g_t = diffuser.noise_coeff(t)
        score = eucl_score_hk(x_t, width=sigma_t)
        
        # Integration step
        if solver_type == 'ODE':
            x_t = x_t + 0.5 * g_t**2 * score * dt
        elif solver_type == 'SDE':
            x_t = x_t + 0.5*g_t**2 * score * dt + g_t * torch.rand_like(x_t) * dt**0.5  # TODO: debug
        else:
            raise NotImplementedError(f'Integration method {solver_type} not supported')
        t -= dt
        trajectories.append(x_t)
    return x_t, trajectories


def _test_denoise():
    batch_size = 8192
    x_1 = sample_euclid_hk(batch_size, t=1.0)
    diffuser = VarianceExpandingDiffusion(sigma=1.1)
    
    num_steps = 200
    x_0, trajectories = denoise_backward(x_1, diffuser, num_steps)
    
    cmap = mpl.colormaps.get_cmap('viridis')
    xs = torch.linspace(-4, 4, 100)
    steps = [20, 50, 150, 195]
    
    fig, axes = plt.subplots(1, len(steps), figsize=(4*len(steps), 4), sharey=True)
    fig.suptitle('Density During Reverse Process')
    axes[0].set_ylabel('Density')
    for ax, step in zip(axes, steps):
        # Check that stats of denoised samples match params of analytical HK
        t = 1 - step/num_steps
        x_t = trajectories[step]
        sigma_t = diffuser.sigma_func(t)
        #assert torch.allclose(torch.std(x_t), sigma_t, atol=5e-2), \
        #    f'StDev of samples {torch.std(x_t).item():.4f} does not match marginal HK StDev {sigma_t:.4f}'
        
        # Display reverse process plots
        ax.set_title(f'$t = {t:.3f}$')
        ax.set_xlabel('$x$')
        hk = euclidean_heat_kernel(xs, t)
        hk /= hk.sum() * (xs[1] - xs[0])
        ax.plot(grab(xs), grab(hk), ls='--', color=cmap(t), label=f'Analytic HK')
        ax.hist(grab(x_t), bins=50, color=cmap(t), alpha=0.55, density=True, label=f'Denoised samples')
        ax.legend(fontsize=7.5)
    fig.show()

_test_denoise()

## Denoising with a Trained Score Network

First we build a very simple MLP as our score network, which will input both $x$ and $t$ (for now, no special embeddings for $t$, just handle the raw time as-is)

In [ ]:
class ScoreNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(2, 8),  # data & time = 1 + 1 dims
            torch.nn.SiLU(),
            torch.nn.Linear(8, 8),
            torch.nn.SiLU(),
            torch.nn.Linear(8, 8),
            torch.nn.SiLU(),
            torch.nn.Linear(8, 1))

    def forward(self, x_t, t):
        if len(t.shape) == 1:
            t = t.unsqueeze(-1)
        return self.net(torch.cat([x_t, t], dim=-1))


def _test_score_net():
    batch_size = 100
    x = torch.randn((batch_size, 1))
    t = torch.rand((batch_size,))
    s_t = ScoreNet()(x, t)
    print('x shape:', x.shape)
    print('t shape:', t.shape)
    print('s_t shape:', s_t.shape)
    assert s_t.shape == x.shape, \
        'Score output should have same shape as input'
    print('[PASSED]')

_test_score_net()

Define the conventional **score matching** loss

In [ ]:
def score_matching_loss(x_0, diffuser, score_net, tol=1e-5):
    t = torch.rand((x_0.size(0),))
    t = tol + (1 - tol) * t  # stability near endpoints
    
    # Diffuse x_0 -> x_t, get s(x_t, t)
    x_t = diffuser(x_0, t.squeeze())
    sigma_t = diffuser.sigma_func(t)[:, None]
    score = score_net(x_t, t) / sigma_t  # rescale score net
    #NOTE: make sure shapes line up!! Can get silent side-effects
    
    # s(x_t, t) should approximate grad log N(x_t; x_0, sigma_t^2)
    eps = (x_t - x_0) / sigma_t
    loss = (sigma_t * score + eps)**2  # weight factor of sigma(t)^2 for stability
    return torch.mean(loss)

Now do the training

In [ ]:
# Diffusion params
sigma = 1.1
diffuser = VarianceExpandingDiffusion(sigma)
score_net = ScoreNet()

# Training hyperparams
lr = 1e-3
epochs = 1000
batch_size = 1024
optimizer = torch.optim.Adam(params=score_net.parameters(), lr=lr)

# Training loop
x_0 = torch.zeros((batch_size, 1))
losses = []
for epoch in tqdm.tqdm(range(epochs)):
    optimizer.zero_grad()
    loss = score_matching_loss(x_0, diffuser, score_net)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch}/{epochs} | Loss = {loss.item():.6f}')
    losses.append(loss.item())

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.plot(losses, lw=0.75)
fig.show()

To generate new samples, we define a function to sample from the diffusion model posterior using our trained score network:

In [ ]:
@torch.no_grad()
def euler_sampler(x_1, score_net, diffuser, num_steps=200, solver_type='ODE', verbose=False):
    score_net.eval()
    batch_size = x_1.size(0)
    
    trajectories = []
    dt = 1 / num_steps
    x_t = x_1.clone()
    t = 1.0
    for step in tqdm.tqdm(range(num_steps)):
        # Get ODE / SDE params
        sigma_t = diffuser.sigma_func(t)
        g_t = diffuser.noise_coeff(t)
        score = score_net(x_t, torch.tensor(t).repeat(batch_size,)) / sigma_t

        # Integration step
        if solver_type == 'ODE':
            x_t = x_t + 0.5 * g_t**2 * score * dt  # ODE Euler step
        elif solver_type == 'SDE':
            x_t = x_t + g_t**2 * score * dt + g_t * torch.rand_like(x_t) * dt**0.5  # SDE step
        else:
            raise NotImplementedError(f'Integration method {solver_type} not supported')
        t -= dt

        # Collect and print metrics
        trajectories.append(x_t)
        if verbose:
            print(f'Step {step}/{num_steps} | x_t = {x_t.mean().item():.6f}')
    return x_t, trajectories

In [ ]:
# Initial data: zeros
num_samples = 4096
x_0 = torch.zeros((num_samples, 1))

# Diffuse forward: x_0 -> x_1
sigma = 1.1
diffuser = VarianceExpandingDiffusion(sigma)
x_1 = diffuser(x_0, t=torch.ones((num_samples,)))

# Denoise backward: x_1 -> x_0'
num_steps = 100
x_0, trajectories = euler_sampler(x_1, score_net, diffuser, num_steps, solver_type='ODE')
print('New x_0:', grab(x_0.mean().item()))

# Plot trajectories
times = [1.0, 0.75, 0.5, 0.25, 0.05]
xs = torch.linspace(-5, 5, 100)
cmap = mpl.colormaps.get_cmap('viridis')

fig, axes = plt.subplots(1, len(times), figsize=(4*len(times), 4), sharey=True)
fig.suptitle('Learned Denoising Process')
axes[0].set_ylabel('Density')
for t, ax in zip(times, axes):
    # Denoised samples
    x_t = trajectories[int(num_steps * (1 - t))]
    ax.hist(grab(x_t), bins=50, density=True, color=cmap(t), alpha=0.65, label='Denoised samples')

    # Analytical heat kernel
    hk = euclidean_heat_kernel(xs, t, width=diffuser.sigma_func(t))
    hk /= hk.sum() * (xs[1] - xs[0])
    ax.plot(grab(xs), grab(hk), color=cmap(t), ls='--', label='Analytic HK')
    ax.set_xlabel(r'$x_t$')
    ax.set_title(f'$t = {t}$')
    ax.legend()
fig.show()

### Include time embedding

What if we use time embedding in our score network? Now we enable the score net to learn a good feature embeddings for the time as well.

In [ ]:
class GaussianFourierProjection(torch.nn.Module):
    def __init__(self, embed_dim, scale=30.0):
        super().__init__()

        weights = torch.randn(embed_dim // 2) * scale
        self.W = torch.nn.Parameter(weights, requires_grad=False)

    def forward(self, x):
        x_proj = x[:, None] * self.W[None, :]

        sinx = torch.sin(2*np.pi * x_proj)
        cosx = torch.cos(2*np.pi * x_proj)

        return torch.cat([sinx, cosx], dim=-1)


def _test_embedding():
    print('[Testing GaussianFourierProjection...]')
    batch_size = 5
    t = torch.rand((batch_size,))
    embed_dim = 64
    embedder = GaussianFourierProjection(embed_dim)
    t_embed = embedder(t)
    assert t_embed.shape == (batch_size, embed_dim), \
        '[FAILED: embedded times must have shape [batch_size, embed_dim]]'
    print('[PASSED]')

_test_embedding()

In [ ]:
class TimeEmbeddedScoreNet(torch.nn.Module):
    def __init__(self, hidden_dim, embed_dim):
        super().__init__()
        self.time_embedder = torch.nn.Sequential(  # embeds times
            GaussianFourierProjection(embed_dim),
            torch.nn.Linear(embed_dim, hidden_dim),
            torch.nn.SiLU(),
        )
        self.encoder = torch.nn.Sequential(  # encodes data
            torch.nn.Linear(1, hidden_dim),
        )
        self.decoder = torch.nn.Sequential(  # decodes data + times
            torch.nn.SiLU(),
            torch.nn.Linear(hidden_dim, 1)
        )
    def forward(self, x_t, t):
        t_emb = self.time_embedder(t)  # [B] -> [B, E] -> [B, H]
        x_enc = self.encoder(x_t)  # [B, 1] -> [B, H]
        y = x_enc + t_emb  # additive embedding [B, H]
        return self.decoder(y)


def _test_score_net():
    batch_size = 100
    x = torch.randn((batch_size, 1))
    t = torch.rand((batch_size,))
    
    s_t = TimeEmbeddedScoreNet(hidden_dim=8, embed_dim=64)(x, t)
    print('x shape:', x.shape)
    print('t shape:', t.shape)
    print('s_t shape:', s_t.shape)
    assert s_t.shape == x.shape, \
        'Score output should have same shape as input'
    print('[PASSED]')


_test_score_net()

In [ ]:
# Diffusion params
sigma = 1.1
diffuser = VarianceExpandingDiffusion(sigma)

# Score network
hidden_dim = 8
embed_dim = 256
score_net = TimeEmbeddedScoreNet(hidden_dim, embed_dim)

# Training hyperparams
lr = 1e-3
epochs = 1000
batch_size = 1024
optimizer = torch.optim.Adam(params=score_net.parameters(), lr=lr)

# Training loop
x_0 = torch.zeros((batch_size, 1))
losses = []
for epoch in tqdm.tqdm(range(epochs)):
    optimizer.zero_grad()
    loss = score_matching_loss(x_0, diffuser, score_net)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch}/{epochs} | Loss = {loss.item():.6f}')
    losses.append(loss.item())

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.plot(losses, lw=0.75)
fig.show()

So the loss curve definitely has a different shape, but doesn't quite seem to converge to as low of a value...

Let's generate new samples again:

In [ ]:
# Initial data: zeros
num_samples = 4096
x_0 = torch.zeros((num_samples, 1))

# Diffuse forward: x_0 -> x_1
sigma = 1.1
diffuser = VarianceExpandingDiffusion(sigma)
x_1 = diffuser(x_0, t=torch.ones((num_samples,)))

# Denoise backward: x_1 -> x_0'
num_steps = 100
x_0, trajectories = euler_sampler(x_1, score_net, diffuser, num_steps, solver_type='ODE')
print('New x_0:', grab(x_0.mean().item()))

# Plot trajectories
times = [1.0, 0.75, 0.5, 0.25, 0.05]
xs = torch.linspace(-5, 5, 100)
cmap = mpl.colormaps.get_cmap('viridis')

fig, axes = plt.subplots(1, len(times), figsize=(4*len(times), 4), sharey=True)
fig.suptitle('Learned Denoising Process')
axes[0].set_ylabel('Density')
for t, ax in zip(times, axes):
    # Denoised samples
    x_t = trajectories[int(num_steps * (1 - t))]
    ax.hist(grab(x_t), bins=50, density=True, color=cmap(t), alpha=0.65, label='Denoised samples')

    # Analytical heat kernel
    hk = euclidean_heat_kernel(xs, t, width=diffuser.sigma_func(t))
    hk /= hk.sum() * (xs[1] - xs[0])
    ax.plot(grab(xs), grab(hk), color=cmap(t), ls='--', label='Analytic HK')
    ax.set_xlabel(r'$x_t$')
    ax.set_title(f'$t = {t}$')
    ax.legend()
fig.show()

Seems like the performance is, at best, no better than without time embedding. Maybe this example is simple enough that just inputting raw times is sufficiently expressive to recover the heat kernel on a single d.o.f?